## Setup and Imports

In [ ]:
%pip install -U langchain-community langchain-chroma langchain-openai

In [2]:
import getpass
import os

# LangChain: [SECRET HIDDEN]
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

# OpenAI: [SECRET HIDDEN]
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········
··········


In [3]:
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.memory import ConversationBufferMemory
from langchain.callbacks import StdOutCallbackHandler

In [4]:
from langchain_openai import ChatOpenAI

# we can explore other models too
model = ChatOpenAI(model="gpt-4o")

## Load Game Logs

In [ ]:
with open('Monopoly_Game_1.txt', 'r') as f:
  LOGS = f.read()

agent_role = "Player 1"
# LOGS

## Initialize LLM Agent

In [7]:
# Conciserompt template that instructs the agent on how to make a move

template = """
  You are playing as {agent_role} in a game. Below is the current game history, including all moves made, the game state, and key events:

  {game_history}

  Objective:
  Make your next move based on your strategy and the game’s rules, maximizing your chance of success.

  Guidelines:
  1. Review the game history to understand the current state and objectives.
  2. Identify constraints or rules limiting your possible actions.
  3. Analyze patterns, threats, or opportunities.


  Instructions:
  1. Reason step-by-step to ensure your action aligns with the game’s rules and strategy.
  2. Finally, provide a single valid move, formatted as an action, with no extra text.
  3. Log a summary of the state of the game after each move
  4. Think about short-term impact and long-term potential risks or advantages.

  What is your next move? Give your final response as NEXT MOVE: <move>
"""

prompt = PromptTemplate(
    input_variables=["agent_role", "game_history"],
    template=template
)

chain = prompt | model

## Simulating the Game

**Major question:**

is there a specific structure of the game logs that we can represent the game state and each move programatically? Or could we use another LLM (such as GPT's function calling) to understand each move and game states?

In [8]:
'''
Process the game history and return the current game state.
This would require a working monopoly simulator

'''
def get_current_game_state(game_history):
    # For now, we'll return a placeholder
    return "Current game state based on the history."

You know what.. Let's just simulate the game on our own!

Below is an unfinished simulation of Monopoly.
UPDATE: Mehar advised that we use this simulation: [Monopoly Simulation](https://github.com/mayankkejriwal/GNOME-p3).

**Thus this implementation is currently abandoned until further notice**

In [9]:
import random

In [10]:
class Dice:
  @staticmethod
  def roll():
    return random.randint(1, 6), random.randint(1, 6)

In [11]:
class Property:
  def __init__(self, name, price, rent, owner=None):
    self.name = name
    self.price = price
    self.rent = rent
    self.owner = owner

In [12]:
class Player:
  def __init__(self, name, cash=1500, board_size=40):
    self.name = name
    self.position = 0  # Start at 'Go'
    self.cash = cash  # Starting cash
    self.properties: List[Property] = []
    self.in_jail = False
    self.board_size = board_size
    self.jail_turns = 0

  def move(self, steps):
    self.position = (self.position + steps) % board_size

In [13]:
class Deck:
  def __init__(self, cards):
    self.cards = cards
    random.shuffle(self.cards)

  def draw_card(self):
    card = self.cards.pop(0)
    self.cards.append(card)  # Place it at the bottom of the deck
    return card


In [14]:
class MonopolyGame:
  def __init__(self, players_names):
    self.players = [Player(name) for name in players_names]
    self.current_player_index = 0
    self.board = self.create_board()
    self.chance_deck = Deck(self.create_chance_cards())
    self.community_chest_deck = Deck(self.create_community_chest_cards())
    self.game_over = False

  def create_board(self):
    # Simplified board with properties
    board = []
    for i in range(40):
      if i in [0, 2, 4, 7, 17, 22, 33, 36]:  # Special spaces
        board.append(None)
      else:
        board.append(Property(f"Property {i}", price=100 + i * 10, rent=10 + i * 2))
    return board

  def create_chance_cards(self):
    return ["Advance to Go", "Go to Jail", "Pay $50", "Receive $50"]

  def create_community_chest_cards(self):
    return ["Bank error in your favor. Receive $200", "Doctor's fees. Pay $50"]

  def get_current_player(self):
    return self.players[self.current_player_index]

  def next_turn(self):
    self.current_player_index = (self.current_player_index + 1) % len(self.players)

  def perform_move(self, player: Player, move):
    if move == "roll_dice":
      die1, die2 = Dice.roll()
      steps = die1 + die2
      player.move(steps)
      print(f"{player.name} rolled {die1} and {die2}, moving to position {player.position}")
      self.handle_landing(player)
    elif move.startswith("buy_property"):
      property = self.board[player.position]
      if property and property.owner is None:
        if player.cash >= property.price:
          player.cash -= property.price
          property.owner = player
          player.properties.append(property)
          print(f"{player.name} bought {property.name}")
        else:
          print(f"{player.name} doesn't have enough cash to buy {property.name}")
    # TODO: Add more actions as needed

  def handle_landing(self, player: Player):
    space = self.board[player.position]
    if isinstance(space, Property):
      if space.owner is None:
        print(f"{player.name} landed on unowned property {space.name}")
      elif space.owner != player:
        rent = space.rent
        player.cash -= rent
        space.owner.cash += rent
        print(f"{player.name} paid ${rent} rent to {space.owner.name}")
    elif space is None:
      print(f"{player.name} landed on a special space.")
    # Add more landing logic as needed

  def represent_game_state(self):
    state = {
      "players": [
        {
          "name": player.name,
          "position": player.position,
          "cash": player.cash,
          "properties": [prop.name for prop in player.properties],
          "in_jail": player.in_jail
        } for player in self.players
      ],
      "current_player": self.get_current_player().name
    }

    return state


### **IDEA (for actual project):**

Keeping track of the game state through a LLM could lead to inconsistencies. We'd like to simulate the game through code.

Use GPT-4o's function calling feature to:
1. Take in current logs, line by line
2. For each move, determine which function(s) to call AND what arguments to pass in
3. Update game state by calling the function(s)

Either way, we must somehow simulate the game in order to evaluate the model's performance.
So that when our agent make a sequence of decisions, we know who is winning

## Simulate Agent

Let's predict some steps!

In [15]:
# Currently not interacting with the game, since it's incomplete
game = MonopolyGame(["Player A", "Player B", "Player C", "Player D"])

In [16]:
# Arbitrarily chosen
game_history = LOGS.split("\n")[0:199]

In [17]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(LOGS)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_texts(texts, embeddings)

# Use docsearch to query game log data
query = "What is the current game information?"
docs = docsearch.similarity_search(query)

# We can then process the retrieved docs to extract information about the game state
# and potentially use it to improve the agent's decision-making process
for doc in docs:
    print(doc.page_content)
    print("=====")

GAME INFORMATION: Printing cash balance and net worth of each player: 
GAME INFORMATION: player 3 has a cash balance of $960.0 and a net worth of $1500.0
GAME INFORMATION: player 4 has a cash balance of $1325.0 and a net worth of $1525.0
GAME INFORMATION: player 2 has a cash balance of $1475.0 and a net worth of $1475.0
GAME INFORMATION: player 1 has a cash balance of $1500.0 and a net worth of $1500.0
GAME ACTION: rolling die...
GAME INFORMATION: dies have come up [5, 5]
MOVE INFORMATION: moving player 4 by 10 relative steps forward
MOVE INFORMATION: player is currently in position Reading Railroad; player is moving to position Pennsylvania Railroad
MOVE INFORMATION: player 4 is on a railroad location, namely Pennsylvania Railroad
MOVE INFORMATION: Pennsylvania Railroad is owned by Bank. The option to buy for player 4 is true
GAME PHASE: post-roll phase for player 4
PLAYER 4: I will attempt to buy Pennsylvania Railroad from the bank.
=====
GAME INFORMATION: Printing cash balance and n

In [18]:
response = chain.invoke(
    {"agent_role": agent_role, "game_history": "\n".join(game_history)}
)
response.content

"To determine the best move, let's analyze the current situation:\n\n### Game State Analysis:\n- **You (Player 1):** \n  - **Current Position:** In Jail/Just Visiting\n  - **Cash Balance:** $1500\n  - **Net Worth:** $1500\n  - **Assets Owned:** None\n\n- **Other Players:**\n  - **Player 2:** Cash $1475, Net Worth $1475, No properties\n  - **Player 3:** Cash $960, Net Worth $1500, Assets: Illinois Avenue, North Carolina Avenue\n  - **Player 4:** Cash $1125, Net Worth $1525, Assets: Reading Railroad, Pennsylvania Railroad\n\n### Observations:\n1. **Properties and Railroads:** \n   - Player 3 is acquiring properties, potentially aiming for a monopoly.\n   - Player 4 is collecting railroads, which can yield substantial rent if more are acquired.\n   \n2. **Financial Position:**\n   - You have the highest cash balance with $1500.\n   - Players 3 and 4 have been actively purchasing properties, which might limit their future purchasing power.\n\n3. **Current Turn:**\n   - It's currently the o

Finally, we add an output parser:

In [36]:
import re

def parse_output(output):
  """Parses the output of the LLM agent to extract the next move.

  Args:
    output: The output string from the LLM agent.

  Returns:
    A dictionary containing the extracted move and any additional information.
  """

  # Adjust the regex to match the "Move:" section 
  # (currently not working the best since format is not specified well in prompt)
  move_match = re.search(r"Move:\*\*\n```(.*?)```", output, re.DOTALL)
  print(move_match)

  if move_match:
    move = move_match.group(1).strip()
    return {"move": move}
  else:
    return {"move": None}


In [37]:
parse_output(response.content)

<re.Match object; span=(1580, 1624), match='Move:**\n```\nPLAYER 1: I am skipping turn\n```'>


{'move': 'PLAYER 1: I am skipping turn'}

## Upcoming Phases (actual project):

### Phase 1: Evaluate Decision Making

**Track Game State:**
   - Use object defined above OR Mehar's repo

**Integrate Agent Actions:**  
   - Allow the LLM agent to input actions (e.g., "roll_dice", "buy_property").  

**Set Metrics for Evaluation:**  
   - Track player balances, assets, and net worth.  
   - Explore win/loss or score metrics for final outcomes.  

(If we're going this way, each simulation would be dependent to the environment itself)

### Phase 2: Run Full Game

- Run multiple games and track key metrics.
